In [30]:
import numpy as np
import pandas as pd
# For visualizations
import matplotlib.pyplot as plt
# For regular expressions
import re
# For handling string
import string
# For performing mathematical operations
import math
import spacy

In [48]:
df = pd.read_csv('spam.csv',encoding="ISO-8859-1")

In [49]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [50]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],inplace=True)
df.rename(columns = {'v1':'Category', 'v2':'Message'}, inplace = True)
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [51]:
df.groupby('Category').describe()

Message                                                             
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later  \
spam         747    653  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [52]:
print('% Imbalanced Data:',747/4825)

% Imbalanced Data: 0.15481865284974095


In [53]:
df = pd.concat([
    df[df['Category']=='ham'].sample(df[df['Category']=='spam'].shape[0]), 
    df[df['Category']=='spam']])

In [54]:
df['Category'].value_counts()

Category
ham     747
spam    747
Name: count, dtype: int64

In [55]:
df['spam'] = df['Category'].apply(lambda x: 1 if x=='spam' else 0)
print(df.shape)
df.head()

(1494, 3)


,Category,Message,spam
1516,ham,I need to come home and give you some good lov...,0
5193,ham,"It's wylie, you in tampa or sarasota?",0
5530,ham,I think that tantrum's finished so yeah I'll b...,0
4599,ham,Hi did u decide wot 2 get 4 his bday if not il...,0
2494,ham,You at mu? You should try to figure out how mu...,0


In [56]:
df.dtypes

Category    object
Message     object
spam         int64
dtype: object

In [60]:
df['Message'] = df['Message'].apply( lambda text: ''.join(c for c in text if c.isalnum() or c.isspace()))

In [61]:
print(df['Message'].head(50))

1516     I need to come home and give you some good lovin
5193                   Its wylie you in tampa or sarasota
5530    I think that tantrums finished so yeah Ill be ...
4599    Hi did u decide wot 2 get 4 his bday if not il...
2494    You at mu You should try to figure out how muc...
2566                        You in your room I need a few
1707    Was doing my test earlier I appreciate you Wil...
2252                  Den wat will e schedule b lk on sun
4484                              What not under standing
2218    Nice talking to you please dont forget my pix ...
2050                  Hi where you You in home or calicut
2404    Please give it 2  or i will pick it up on Tues...
2561                              And maybe some pressies
4382      No need lar i go engin Cos my sis at arts today
4954    Doing my masters When will you buy a bb cos i ...
1185                                     Annoying isnt it
1209                           His bday real is in april 
326           

SpaCy for the removal of stopwords and lemmatization. It is a library for advanced Natural Language Processing in Python and Cython.

In [62]:
# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

In [63]:
# Lemmatization with stopwords removal
df['Message']=df['Message'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))


In [64]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Message'],df['spam'], stratify=df['spam'])

In [65]:
X_train.head()

4966    donate å250 UNICEFs Asian Tsunami disaster sup...
3756    yes place town meet exciting adult single UK T...
3174    UR go 2 bahamas CallFREEFONE 08081560665 speak...
168             great escape fancy bridge need lager tomo
5381                            1 new message 02070836089
Name: Message, dtype: object

In [75]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text

In [78]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",
    trainable=True)

2023-07-02 04:02:44.842016: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-07-02 04:02:44.843305: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-07-02 04:02:44.843662: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-07-02 04:02:44.844847: I tensorflow/core/

TypeError: this __dict__ descriptor does not support '_DictWrapper' objects

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:
c=get_sentence_embeding(['mango','banana','Narendra modi','data analytics','natural language processing'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([c[0]],[c[1]])

In [ ]:
#Functional Model

# Bert layers
#Text input passed to bert_preprocess
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
#Processed text passed to bert_encoder
outputs = bert_encoder(preprocessed_text)

# Neural network layers

#Dropout
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

In [ ]:
len(X_train)

In [ ]:
METRICS = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')]

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
#If probablity greater than 0.5 than make it 1 otherwise 0.
#If y_predicted=1->spam
#else:Not spam
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_predicted)
cm 

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')